In [1]:
import qnorm
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import HTML, display, Markdown
from matplotlib.gridspec import GridSpec
from matplotlib_venn import venn2
from maayanlab_bioinformatics.normalization import zscore_normalize, log2_normalize
from maayanlab_bioinformatics.dge import limma_voom_differential_expression
from maayanlab_bioinformatics.harmonization.ncbi_genes import ncbi_genes_lookup
import os
lookup = ncbi_genes_lookup()

import sys
import contextlib
@contextlib.contextmanager
def suppress_output(stdout=True, stderr=True, dest='/dev/null'):
    ''' Usage:
    with suppress_output():
        print('hi')
    '''
    dev_null = open(dest, 'a')
    if stdout:
        _stdout = sys.stdout
        sys.stdout = dev_null
    if stderr:
        _stderr = sys.stderr
        sys.stderr = dev_null
    try:
        yield
    finally:
        if stdout:
            sys.stdout = _stdout
        if stderr:
            sys.stderr = _stderr

# Target Identification

Process all 13 replicatively senescent (RS) samples against a single background, saving the significantly differentially expressed candidates.

For the background dataset, use either "gtex-transcript-stats.tsv" or "archs4-transcript-extra-stats.tsv".

(Note: as of 8/24/2022 there was an issue with processing targets against the most updated "archs4-transcript-stats.tsv" dataset. Despite large log-FC, adjusted p-values were non-significant.)

In [4]:
home = "/Users/edendeng/MaayanLab/target-identifier-proteomics-lncrnas/final/cell-aging-revisions/data/RS_transcripts"
df_transcript_gene_map = pd.read_csv("s3://storage/Tumor_Gene_Target_Screener/transcript-gene-map.tsv.gz", storage_options=dict(client_kwargs=dict(endpoint_url="https://appyters.maayanlab.cloud"), anon=True), sep='\t', header=0, index_col=0, compression='gzip')

# Load background dataset
df_bg_stats = pd.read_csv("s3://storage/Tumor_Gene_Target_Screener/archs4-transcript-extra-stats.tsv", storage_options=dict(client_kwargs=dict(endpoint_url="https://appyters.maayanlab.cloud"), anon=True), sep='\t', index_col=[0,1])
df_bg_transcripts = df_bg_stats.unstack().index.map(lambda idx: idx.partition('.')[0])
df_bg_stats = df_bg_stats.unstack().groupby(df_bg_transcripts, observed=True).sum().stack()
df_bg_expr = df_bg_stats.loc[(slice(None), ['25%', '50%', '75%']), :].unstack()

for file in os.listdir(home):
    # Load RS RNA-seq expression data
    df_expr = pd.read_csv("{}/{}".format(home, file), sep='\t', index_col=0)
    rs = file.split(sep=".tsv")[0]
    df_expr_transcripts = df_expr.index.map(lambda idx: idx.partition('.')[0])
    df_expr = df_expr.groupby(df_expr_transcripts, observed=True).sum()
    
    # Distribution matching between RS samples & the background
    common_index = list(set(df_expr.index) & set(df_bg_expr.index))
    target_distribution = df_bg_expr.loc[common_index, :].median(axis=1)
    df_expr_norm = qnorm.quantile_normalize(df_expr.loc[common_index, :], target=target_distribution)
    df_bg_expr_norm = qnorm.quantile_normalize(df_bg_expr.loc[common_index, :], target=target_distribution)
    
    # Perform differential expression between samples & the background
    with suppress_output():
        dge = limma_voom_differential_expression(
            df_bg_expr_norm, df_expr_norm,
            voom_design=True,
        )

        dge['ensembl_transcript_id'] = dge.index
        dge['gene_symbol'] = df_transcript_gene_map.loc[dge.index, 'gene_symbol'].apply(lambda g: lookup(g) or g)
        dge['label'] = dge.apply(lambda r: f"{r['ensembl_transcript_id']} - {r['gene_symbol']}", axis=1)
    
    # Narrow down candidate set
    dge['-log(adj.P.Val)'] = -np.log(dge['adj.P.Val'])
    prod = (np.abs(dge['t']) * dge['logFC'])
    dge['is_deg'] = dge['adj.P.Val'] < 0.05
    dge['is_significant'] = prod > prod.mean() + 3 * prod.std()
    dge['score1'] = dge['is_deg'].astype(int) + dge['is_significant'].astype(int)
    
    # Save results
    path1 = "/Users/edendeng/MaayanLab/target-identifier-proteomics-lncrnas/final/cell-aging-revisions/results/RS/transcripts/all/ARCHS4_extra/{}_candidates.csv".format(rs)
    dge[dge.score1 >= 2].sort_values(['score1', 't'], ascending=False).to_csv(path1)

# Compile Results

For each background, aggregate all the significant targets. For the GTEx targets, we filter out DEXI, which appears to be an artifact of the background data.

In [7]:
data = []
home = "/Users/edendeng/MaayanLab/target-identifier-proteomics-lncrnas/final/cell-aging-revisions/results/RS/transcripts/all/GTEx"
for file in os.listdir(home):
    f = pd.read_csv("{}/{}".format(home, file), index_col=0)
    rs = file.split(sep="_candidates")[0]
    data.append([ rs, *f.label[f.gene_symbol != 'DEXI'] ])
    #data.append([ rs, *f.label ])

pd.DataFrame(data).T.to_csv("/Users/edendeng/MaayanLab/target-identifier-proteomics-lncrnas/final/cell-aging-revisions/results/gtex_all_transcripts.csv",
                            index=False, header=False)